## Load libraries

In [1]:
import numpy as np
import pandas as pd
import math
import pickle
import json
from gurobipy import *

## Functions

In [2]:
def pickle_load(file_dir):
  with open(file_dir, "rb") as fp:   # Unpickling
    return pickle.load(fp)

def json_load(file_dir):
    with open(file_dir) as f:
        return json.load(f)

In [3]:
data_dir = './data/'

# scenario number & distribution settings
scenario_num = 1000
distribution_choice = 'normal'

# Sets directory
Bset_dir = data_dir + 'Bset.txt'
Bij_set_dir = data_dir + 'Bij_set.txt'
Sset_dir = data_dir+'Sset_' + str(scenario_num) + '.txt'

deterministic_params_dir = data_dir + 'deterministic_params.json'
capacity_i_dir = data_dir + 'capacity_i.json'
capacity_i_constant_dir = data_dir + 'capacity_i_constant.json'

p_s_dir = data_dir + 'p_s_' + str(scenario_num) + '.json'
demScens_dir = data_dir + 'demScens_' + distribution_choice + '_' + str(scenario_num) + '.dict'

In [4]:
# Sets
Bset = pickle_load(Bset_dir)                                  # 22 Bike Stations
Bij_set = pickle_load(Bij_set_dir)                                  # Bike pairs                                        
Sset = pickle_load(Sset_dir)                                     # Scenario sets
deterministic_params = json_load(deterministic_params_dir)       # deterministic parameters

c = deterministic_params['c']                            # unit procurement cost
v_i = deterministic_params['v_i']                               # stock-out cost
w_i = deterministic_params['w_i']                              # time-waste cost 
t_ij = deterministic_params['t_ij']                    # unit transshipment cost 
capacity_i = json_load(capacity_i_dir)
p_s = json_load(p_s_dir)
demScens = pickle_load(demScens_dir)

In [5]:
cpy = {}
for pair, val in demScens.items():
    cpy[((pair[0], pair[1]), pair[2])] = val
demScens = cpy

The formulation of the two-stage stochastic program problem i written as follows:

<b>First stage varibles:</b> <br>
$$x_i: \text{the number of bikes to assign to bike-station i} \in \text{B at the beginning of the service}$$
<b>Second stage variables: </b>
$$
\beta_{ijs}: \text{Number of rented bikes from bike-station i to bike-station j in scenario s}\\
I_{is}^{+}: \text{Realized surplus of bikes at bike-station i in scenario s} \\
I_{ijs}^{-}: \text{Realized shortage of bikes at origin-destination pair i, j in scenario s}\\
\rho_{ijs}: \text{Number of redirected bikes from bike-station i to bike-station j in scenario s (in case of overflow)}\\
O_{is}^{+}: \text{Residual capacity at bike-station i in scenario s} \\
O_{is}^{-}: \text{Overflow at bike-station i in scenario s} \\
\tau_{ijs}: \text{Number of transshipped bikes from bike-station i to bike-station j in scenario s} \\
T_{is}^{+}: \text{Excess of bikes at bike-station i in scenario s} \\
T_is^{-}: \text{Lack of bikes at bike-station i in scenario s} \\
$$

<b>Formulation:</b>
$$
\begin{align*}
    \text{minimize: }   & c \sum_{i}^B x_i + \sum_{s=1}^S p_s\sum_{i=1}^B [v_i\sum_{j=1}^B I_{ijs}^{-} + w_iO_{is}^{-} + \sum_{j=1}^B t_{ij}\tau_{ijs}]\\
    \text{subject to: }   
        & x_{i} \le k_i, \forall i \in B &\text{(Bike Station Capacity)}\\
        & \beta_{ijs} = \xi_{ijs} - I_{ijs}^{-} , \forall i,j \in B, s \in S &\text{(rented bike successfully made from station i to j)}\\
        & I_{is}^{+} - \sum_{j=1}^B I_{ijs}^{-} = x_i - \sum_{j=1}^B \xi_{ijs} , \forall i \in B, s \in S &\text{(realized surplus and shortage)}\\
        & O_{is}^{+} - O_{is}^{-} = k_i - x_i + \sum_{j=1}^B \beta_{ijs} - \sum_{j=1}^B \beta_{ijs}, \forall i \in B, s \in S &\text{(residual capacity and overflow after rental)}\\
        & \sum_{j=1}^{B} \rho_{ijs} = O_{is}^{-}, \forall i \in B, s \in S &\text{(Redirection identifies station overflow)}\\
        & \sum_{j=1}^{B} \rho_{jis} \le O_{is}^{+}, \forall i \in B, s \in S &\text{(Sucessful redirection less than residual capacity)}\\
        & T_{is}^{+} - T_{is}^{-} = k_i - O_{is}^{+} + \sum_{j=1}^{B}\rho_{jis} - x_i, \forall i \in B, s \in S &\text{(Excess and Lack of bikes at stations)}\\
        & \sum_{j=1}^{B}\tau_{ijs} = T_{is}^{+}, \forall i \in B, s \in S &\text{(Transshipment of excessive bikes)}\\
        & \sum_{j=1}^{B}\tau_{jis} = T_{is}^{-}, \forall i \in B, s \in S &\text{(Transshipment fulfillment)}\\
        & x_i, I_{is}^{+}, O_{is}^{+}, O_{is}^{-}, T_{is}^{+}, T_{is}^{-} \in \mathbb{Z}^{+}, \forall i \in B, s \in S\\
        & \tau_{ijs}, \beta_{ijs}, \rho_{ijs}, I_{ijs}^{-} \in \mathbb{Z}^{+}, , \forall i,j \in B, s \in S\\
\end{align*}
$$

## Building Model

In [6]:
m = Model("2SP_ExtForm")

Using license file /Users/ruijian/gurobi.lic
Academic license - for non-commercial use only


### Set variables

In [7]:
# first stage variables
x = m.addVars(Bset, vtype=GRB.INTEGER, name='x')

# second stage variables
beta_ijs = m.addVars(Bset,Bset,Sset, vtype=GRB.INTEGER, name='beta_ijs')
I_is_surplus = m.addVars(Bset,Sset, vtype=GRB.INTEGER, name='I_ijs_surplus')
I_ijs_shortage = m.addVars(Bset,Bset, Sset, vtype=GRB.INTEGER, name='I_ijs_shortage')
rho_ijs = m.addVars(Bset,Bset, Sset, vtype=GRB.INTEGER, name='rho_ijs')
O_is_resCap = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='O_is_resCap')
O_is_overF = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='O_is_overF')
tau_ijs = m.addVars(Bset,Bset, Sset, vtype=GRB.INTEGER, name='tau_ijs')
T_is_excess = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='T_is_excess')
T_is_lack = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='T_is_lack')

m.setObjective(c*quicksum(x[i]for i in Bset) \
               + quicksum(p_s[s]*quicksum(v_i * I_ijs_shortage.sum(i,'*',s)\
                                          + w_i * O_is_overF[(i,s)] \
                                          + t_ij * tau_ijs.sum(i, '*', s) for i in Bset)for s in Sset) )

### Set modelsense

In [8]:
m.modelSense = GRB.MINIMIZE

### Set constraints

In [9]:
m.addConstrs(
    (x[i] <= capacity_i[i] for i in Bset), name='assignment_capacity');

In [10]:
m.addConstrs(
    (beta_ijs[i, j, s] == demScens[(i, j), s] - I_ijs_shortage[i, j, s] for i in Bset for j in Bset for s in Sset), name='actual_rental');

In [11]:
m.addConstrs(
    (I_is_surplus[i,s] - I_ijs_shortage.sum(i, '*', s) == x[i] - sum(demScens[(i,j),s] for j in Bset) for i in Bset for s in Sset), \
        name='realized_surplus_shortage');

In [12]:
m.addConstrs((O_is_resCap[i,s] - O_is_overF[i,s] \
              == capacity_i[i] - x[i] + beta_ijs.sum(i, '*', s) - beta_ijs.sum('*', i, s) for i in Bset for s in Sset),
            name='residual_overflow_after_rental');

In [13]:
m.addConstrs((rho_ijs.sum(i, '*', s) ==  O_is_overF[i, s] for i in Bset for s in Sset),
            name='redirecting_overflow');

In [14]:
m.addConstrs((rho_ijs.sum('*', i, s) <=  O_is_resCap[i, s] for i in Bset for s in Sset),
            name='successful_redictions');

In [15]:
m.addConstrs((T_is_excess[i, s] - T_is_lack[i, s] \
              == capacity_i[i] - O_is_resCap[i, s] + rho_ijs.sum('*', i, s) - x[i] for i in Bset for s in Sset),
            name='successful_redictions');

In [16]:
m.addConstrs((tau_ijs.sum(i, '*', s) ==  T_is_excess[i, s] for i in Bset for s in Sset),
            name='transshipment_excessive_bike');

In [17]:
m.addConstrs((tau_ijs.sum('*', i, s) ==  T_is_lack[i, s] for i in Bset for s in Sset),
            name='transshipment_fulfillment');

In [18]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 638022 rows, 2046022 columns and 5082022 nonzeros
Model fingerprint: 0xaf41c830
Variable types: 0 continuous, 2046022 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-06, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 550022 rows and 846628 columns (presolve time = 5s) ...
Presolve removed 550022 rows and 868628 columns (presolve time = 11s) ...
Presolve removed 550022 rows and 868628 columns (presolve time = 16s) ...
Presolve removed 550022 rows and 868628 columns
Presolve time: 16.95s
Presolved: 88000 rows, 1177394 columns, 3136116 nonzeros
Variable types: 0 continuous, 1177394 integer (231372 binary)
Found heuristic solution: objective 925.6942980

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.13s

Barrier statis

In [19]:
vs = m.ObjVal
vs

564.3325144999928

In [20]:
x

{'A': <gurobi.Var x[A] (value 10.0)>,
 'B': <gurobi.Var x[B] (value 10.0)>,
 'C': <gurobi.Var x[C] (value 10.0)>,
 'D': <gurobi.Var x[D] (value 10.0)>,
 'E': <gurobi.Var x[E] (value 10.0)>,
 'F': <gurobi.Var x[F] (value 10.0)>,
 'G': <gurobi.Var x[G] (value 10.0)>,
 'H': <gurobi.Var x[H] (value 8.0)>,
 'I': <gurobi.Var x[I] (value 8.0)>,
 'J': <gurobi.Var x[J] (value 10.0)>,
 'K': <gurobi.Var x[K] (value 9.0)>,
 'L': <gurobi.Var x[L] (value 10.0)>,
 'M': <gurobi.Var x[M] (value 8.0)>,
 'N': <gurobi.Var x[N] (value 10.0)>,
 'O': <gurobi.Var x[O] (value 10.0)>,
 'P': <gurobi.Var x[P] (value 10.0)>,
 'Q': <gurobi.Var x[Q] (value 10.0)>,
 'R': <gurobi.Var x[R] (value 10.0)>,
 'S': <gurobi.Var x[S] (value 8.0)>,
 'T': <gurobi.Var x[T] (value 10.0)>,
 'U': <gurobi.Var x[U] (value 10.0)>,
 'V': <gurobi.Var x[V] (value 9.0)>}

## VSS

In [21]:
# Calculate the mean of the demand
demand_mean = {}
for pair, val in demScens.items():
    station_pair, scenario = pair
    if station_pair in demand_mean:
        demand_mean[station_pair] += p_s[scenario] * val
    else:
        demand_mean[station_pair] = p_s[scenario] * val
for key, val in demand_mean.items():
    demand_mean[key] = int(val)

In [22]:
# Calculate the optimal x^* with mean demand
M = Model("1 Scenairo Model")
M.modelSense = GRB.MINIMIZE

### Solve for optimal $x_{MV}^*$

In [23]:
# first stage variables
x_M = M.addVars(Bset, vtype=GRB.INTEGER, name='x_M')

# second stage variables
M.Params.outputFlag = 0  # turn off output
M.params.logtoconsole=0  # turn off logging of process 


beta_ij = M.addVars(Bset, Bset, name='beta_ij')
I_i_surplus = M.addVars(Bset, name='I_ij_surplus')
I_ij_shortage = M.addVars(Bset,Bset, name='I_ij_shortage')
rho_ij = M.addVars(Bset,Bset, name='rho_ij')
O_i_resCap = M.addVars(Bset, name='O_i_resCap')
O_i_overF = M.addVars(Bset, name='O_i_overF')
tau_ij = M.addVars(Bset,Bset, name='tau_ij')
T_i_excess = M.addVars(Bset, name='T_i_excess')
T_i_lack = M.addVars(Bset, name='T_i_lack')

M.setObjective(c*quicksum(x_M[i]for i in Bset) + quicksum(v_i * I_ij_shortage.sum(i,'*') + w_i * O_i_overF[i] + t_ij * tau_ij.sum(i, '*') for i in Bset))

In [24]:
M.addConstrs(
    (x_M[i] <= capacity_i[i] for i in Bset), name='assignment_capacity');

In [25]:
actual_rental = {}
for ij in Bij_set:
    actual_rental[ij] = M.addConstr(beta_ij[ij] + I_ij_shortage[ij] == demand_mean[ij], name="actual_rental_"+str(ij))

In [26]:
realized_surplus_shortage = {}
for i in Bset:
    realized_surplus_shortage[i] = M.addConstr(-I_i_surplus[i] + I_ij_shortage.sum(i, '*') + x_M[i] \
                                               == sum(demand_mean[(i,j)] for j in Bset), \
                                               name="realized_surplus_shortage_"+str(i))                                               

In [27]:
M.addConstrs((O_i_resCap[i] - O_i_overF[i] \
              == capacity_i[i] - x_M[i] + beta_ij.sum(i, '*') - beta_ij.sum('*', i) for i in Bset),
            name='residual_overflow_after_rental');

In [28]:
M.addConstrs((rho_ij.sum(i, '*') ==  O_i_overF[i] for i in Bset),
            name='redirecting_overflow');

In [29]:
M.addConstrs((rho_ij.sum('*', i) <=  O_i_resCap[i] for i in Bset),
            name='successful_redictions');

In [30]:
M.addConstrs((T_i_excess[i] - T_i_lack[i] \
              == capacity_i[i] - O_i_resCap[i] + rho_ij.sum('*', i) - x_M[i] for i in Bset),
            name='successful_redictions');

In [31]:
M.addConstrs((tau_ij.sum(i, '*') ==  T_i_excess[i] for i in Bset),
            name='transshipment_excessive_bike');

In [32]:
M.addConstrs((tau_ij.sum('*', i) ==  T_i_lack[i] for i in Bset),
            name='transshipment_fulfillment');

In [33]:
M.optimize()

In [34]:
x_M['A']

<gurobi.Var x_M[A] (value -0.0)>

### Solve Extensive Model with $x^*_{MV}$

In [35]:
for key, val in x_M.items():
    x[key].lb = val.X
    x[key].ub = val.X
m.update()
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 638022 rows, 2046022 columns and 5082022 nonzeros
Model fingerprint: 0xfa2cb21d
Variable types: 0 continuous, 2046022 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-06, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 572022 rows and 1100022 columns (presolve time = 5s) ...
Presolve removed 637956 rows and 2045076 columns
Presolve time: 9.74s
Presolved: 66 rows, 946 columns, 2376 nonzeros
Variable types: 0 continuous, 946 integer (0 binary)
Found heuristic solution: objective 925.6942980

Explored 0 nodes (0 simplex iterations) in 12.40 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 925.694 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.256942980000e+02, best bound 9.256942980000e+02, gap 0.0000%


In [36]:
print("VSS: {}".format(m.objVal - vs))

VSS: 361.36178350004684


## EVPI

In [37]:
demand_for_each_scenario = {scenario: {} for scenario in Sset}
for pair, val in demScens.items():
    station_pair, scenario = pair
    demand_for_each_scenario[scenario][station_pair] = val

In [38]:
EVPI_val = 0
for scenario, demand in demand_for_each_scenario.items():
    for ij in Bij_set:
        actual_rental[ij].rhs = demand[ij]
    for i in Bset:
        realized_surplus_shortage[i].rhs = sum(demand[(i,j)] for j in Bset)
    M.update()
    M.optimize()
    EVPI_val += p_s[scenario] * M.objVal

In [39]:
print("EVPI: {}".format(vs - EVPI_val))

EVPI: 39.26793849999308
